In [48]:
import pandas as pd
import numpy as np
import os
from glob import glob
import random
import matplotlib.pylab as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator
import keras
import tensorflow as tf

In [49]:
def flatten(dimData, images):
    images = np.array(images)
    images = images.reshape(len(images), dimData)
    images = images.astype('float32')
    images /=255
    return images


In [75]:
working_dir = r'E:\ML\pro3\Dataset\dataset'
os.chdir(working_dir)
img_size =32

In [ ]:
folders, labels, images = [ 'Male', 'Female'], [], []
for folder in folders:
    print(folder)
    for path in os.listdir(os.getcwd()+'/'+folder):
        img = cv2.imread(folder+'/'+path,0)
        #cv2.imshow('img', img)
        #cv2.waitKey(1)
        images.append(cv2.resize(img, (img_size, img_size)))
        labels.append(folders.index(folder))

Male


In [ ]:
#to_train= 0
train_images, test_images, train_labels, test_labels = [],[],[],[]
for image, label in zip(images, labels):
    if to_train<5:
        train_images.append(image)
        train_labels.append(label)
        to_train+=1
    else:
        test_images.append(image)
        test_labels.append(label)
        to_train = 0


In [ ]:
dataDim = np.prod(images[0].shape)
train_data  = flatten(dataDim, train_images)
test_data = flatten(dataDim, test_images)

In [ ]:
from keras.utils import to_categorical
train_labels = np.array(train_labels)

test_labels = np.array(test_labels)
train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)


In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
       keras.layers.Dense(2, activation='softmax')
])

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels_one_hot, batch_size = 256, epochs=10, verbose=1,
                    validation_data=(test_data, test_labels_one_hot))

In [ ]:
[test_loss, test_acc] = model.evaluate(test_data )
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))


In [ ]:
import matplotlib.pyplot as plt
#plotting the Accuracy of test and training sets
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
rounded_predictions = model.predict_classes(test_data, batch_size=128, verbose=0)

rounded_labels=np.argmax(test_labels_one_hot , axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(rounded_labels, rounded_predictions)
cm
import seaborn as sns
figure = plt.figure(figsize=(12, 6))
sns.heatmap(cm, annot=True,cmap=plt.cm.cool,fmt='d')
plt.tight_layout()
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

nn_fpr_keras, nn_tpr_keras, nn_thresholds_keras = roc_curve(rounded_labels  , rounded_predictions)
auc_keras = auc(nn_fpr_keras, nn_tpr_keras)
plt.plot(nn_fpr_keras, nn_tpr_keras, marker='.', label='Neural Network (auc = %0.3f)' % auc_keras)


In [ ]:

model.save(r'E:\ML')

In [ ]:
2